# Day 1 - Lab 2: Generating a Product Requirements Document (PRD)

**Objective:** Use the structured `day1_user_stories.json` artifact from the previous lab to generate a formal, comprehensive Product Requirements Document (PRD) in markdown format.

**Estimated Time:** 60 minutes

**Introduction:**
With a validated set of user stories, we can now create a higher-level planning document: the PRD. A PRD serves as the source of truth for the product team, outlining the project's purpose, features, and requirements. In this lab, you will use an LLM to synthesize the detailed user stories into this formal document.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

This initial block sets up our environment. It adds the project's root directory to the Python path, allowing us to import our custom `utils.py` script. We then initialize the connection to our Large Language Model (LLM) and load the JSON artifact from the previous lab.

**Model Selection:**
You can change the `model_name` parameter in the `setup_llm_client()` function to any of the models listed in `utils.py`, such as `"gemini-2.5-flash"` or `"meta-llama/Llama-3.3-70B-Instruct"`.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the user stories JSON file and the PRD template.
- `save_artifact()`: To save our generated PRD and Pydantic model.

In [1]:
import sys
import os
import json

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    # Assumes the notebook is in 'labs/Day_01_.../'
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, prompt_enhancer

# Initialize the LLM client. You can change the model here.
client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the artifact from Lab 1
user_stories_data = load_artifact("day1_user_stories.json")
# if user_stories_str:
#     user_stories_data = json.loads(user_stories_str)
# else:
#     user_stories_data = []

2025-10-28 14:06:48,588 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating a Simple PRD

**Task:** Use the loaded user stories to generate a simple PRD.

**Instructions:**
1. Create a prompt that instructs the LLM to act as a Product Manager.
2. Provide the `user_stories_data` as context.
3. Ask the LLM to generate a PRD with three sections: "Introduction", "User Personas", and "Features / User Stories".

**Expected Quality:** A clean markdown document that correctly summarizes the provided user stories into the requested sections.

In [2]:
# TODO: Write a prompt to generate a simple PRD.
simple_prd_prompt = f"""
Write a simple Product Requirements Document (PRD) based on the following user stories:{user_stories_data}
"""
enhanced_prd_prompt = prompt_enhancer(simple_prd_prompt)

print("--- Generating Simple PRD ---")
if user_stories_data:
    simple_prd_output = get_completion(simple_prd_prompt, client, model_name, api_provider)
    print(simple_prd_output)
else:
    print("Skipping PRD generation because user stories are missing.")

2025-10-28 14:06:49,222 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


--- Generating Simple PRD ---
Of course. Here is a simple Product Requirements Document (PRD) based on the user stories provided.

---

## **Product Requirements Document: Project OnboardFlow**

*   **Version:** 1.0
*   **Date:** October 26, 2023
*   **Status:** Draft
*   **Author:** Product Management

### 1. Introduction & Problem Statement

Our current employee onboarding process is manual, inconsistent, and often leaves new hires feeling unprepared and managers overwhelmed. This leads to decreased productivity in the first few months, a disjointed new hire experience (especially for remote employees), and significant administrative overhead for managers, HR, and IT.

**Project OnboardFlow** is a unified platform designed to automate, personalize, and streamline the onboarding experience from the moment an offer is signed through the first 90 days. It will provide a structured, engaging, and efficient journey for new hires while giving managers the visibility and tools they need to 

### Challenge 2 (Intermediate): Generating a PRD from a Template

**Task:** Instead of just listing sections, we will now provide the LLM with a formal template to ensure the PRD's structure is consistent and complete.

**Instructions:**
1. First, load the contents of `templates/prd_template.md` into a variable.
2. Create a new prompt that instructs the LLM to act as a Senior Product Manager.
3. Provide both the `user_stories_data` and the `prd_template_content` as context.
4. Instruct the LLM to populate the template with the information from the user stories, ensuring every section of the template is filled out.

> **Tip:** The template has sections like 'Success Metrics' and 'Out of Scope' that aren't in the user stories. This is your chance to guide the LLM's creativity! Instruct it to infer logical content for these sections based on the project's overall goal.

**Expected Quality:** A complete PRD that strictly follows the structure of the provided template file, demonstrating the LLM's ability to perform structured content generation.

In [3]:
# Load the PRD template
prd_template_content = load_artifact("templates/prd_template.md")

# TODO: Write a prompt to populate the PRD template.
template_prd_prompt = f"""
Act as a Senior Product Manager. Using the following user stories, populate the provided PRD template.
User Stories: {user_stories_data}
PRD Template:{prd_template_content}
"""

print("--- Generating PRD from Template ---")
if user_stories_data and prd_template_content:
    prd_from_template_output = get_completion(template_prd_prompt, client, model_name, api_provider)
    print(prd_from_template_output)
else:
    print("Skipping PRD generation because user stories or template are missing.")
    prd_from_template_output = ""

--- Generating PRD from Template ---
Of course. As a Senior Product Manager, I've synthesized the provided user stories into a comprehensive Product Requirements Document. Here is the populated template.

***

# Product Requirements Document: Project Ignition: The New Hire Onboarding Platform

| Status | **Draft** |
| :--- | :--- |
| **Author** | Senior Product Manager |
| **Version** | 1.0 |
| **Last Updated** | October 26, 2023 |

## 1. Executive Summary & Vision
*A high-level overview for stakeholders. What is this product, why are we building it, and what is the ultimate vision for its success?*

Project Ignition is a centralized, automated platform designed to streamline and enrich the new hire onboarding experience. We are building this to solve the current fragmented and manual onboarding process, which leads to administrative overhead, new hire anxiety, and delayed productivity. The platform will serve new hires, their managers, and internal support teams like HR and IT.

Our v

### Challenge 3 (Advanced): Programmatic Validation with Pydantic

**Task:** We will now create a Pydantic model to represent the structure of our PRD. This allows us to programmatically validate any PRD, ensuring it meets our standards before it's accepted as a formal artifact.

**Instructions:**
1.  Prompt the LLM to generate a Pydantic model that reflects the structure of the `prd_template.md`. The model should have fields for each major section (e.g., `introduction: str`, `user_personas: List[str]`, `user_stories: List[Dict]`).
2.  Save this generated model code to a file named `app/validation_models/prd_model.py`.
3.  While we won't write the full validation script in this lab, generating the Pydantic model itself is the key advanced step. It creates a reusable, code-based standard for our documentation.

**Expected Quality:** A Python file containing a valid Pydantic model that can be used in the future to validate PRD documents automatically. This represents a shift from manual document review to automated governance.

In [ ]:
# TODO: Write a prompt to generate a Pydantic model for the PRD.
# Tip: Be specific. Tell the LLM to create a class named 'ProductRequirementsDocument' and to use appropriate types from Python's 'typing' library.
pydantic_model_prompt = f"""
Create a Pydantic model named 'ProductRequirementsDocument' that represents the structure of the following Product Requirements Document (PRD). Use appropriate types from Python's 'typing' library.
PRD Content:{prd_template_content}
"""

print("--- Generating Pydantic Model for PRD ---")
if prd_template_content:
    pydantic_model_code = get_completion(pydantic_model_prompt, client, model_name, api_provider)
    
    # Clean up the code if it's wrapped in markdown fences
    if '```' in pydantic_model_code:
        pydantic_model_code = pydantic_model_code.split('```')[1].lstrip('python').strip()
    
    print("\n--- Generated Pydantic Model ---")
    print(pydantic_model_code)

    # Save the generated Pydantic model code to a file.
    model_path = "app/validation_models/prd_model.py"
    save_artifact(pydantic_model_code, model_path, overwrite=True)
else:
    print("Skipping Pydantic model generation because template is missing.")

# Finally, save the completed PRD from the intermediate challenge
if prd_from_template_output:
    save_artifact(prd_from_template_output, "day1_prd.md")

--- Generating Pydantic Model for PRD ---

--- Generated Pydantic Model ---
import datetime
from enum import Enum
from typing import List, Optional

from pydantic import BaseModel, Field


# --- Enums and Helper Models ---

class DocumentStatus(str, Enum):
    """Enumeration for the PRD's status."""
    DRAFT = "Draft"
    IN_REVIEW = "In Review"
    APPROVED = "Approved"


class ProblemDefinition(BaseModel):
    """Defines the problem the product aims to solve."""
    problem_statement: str = Field(..., description="A clear and concise description of the primary problem.")
    user_personas: List[str] = Field(..., description="A list of key user personas affected by the problem.")


class GoalMetric(BaseModel):
    """Represents a single goal with its corresponding KPI and target."""
    goal: str = Field(..., description="The high-level objective.")
    kpi: str = Field(..., description="The Key Performance Indicator to measure progress towards the goal.")
    target: str = Field(...

ArtifactError: Artifact already exists: C:\Workspace\AG-AISOFTDEV\artifacts\app\validation_models\prd_model.py. Pass overwrite=True to replace.

## Lab Conclusion

Excellent work! You have now taken the structured user stories from the first lab and synthesized them into a formal Product Requirements Document. You also created a Pydantic model to enforce the structure of this document, introducing automated governance into your workflow. The `day1_prd.md` artifact will be the primary input for Day 2, where we will begin designing our system's architecture and database.

> **Key Takeaway:** Using an LLM to populate a pre-defined template is a powerful pattern for creating consistent, high-quality documentation at scale. It combines the LLM's language skills with your required structure.